This Jupyter notebook contains normal ChromaDB Queries without LLM

In [2]:
import chromadb
from sentence_transformers import SentenceTransformer

# === CONFIGURATION ===
CHROMA_DB_DIR = '../chroma_db'
COLLECTION_NAME = 'government_schemes_chunks'
TOP_K = 15

# Fields Setup
CORE_FIELDS = {"Description"}
LONG_FIELDS = {"Application Process", "Eligibility Criteria", "Documents Required", "Benefits"}

# === Initialize Clients (Call Once) ===
client = chromadb.PersistentClient(path=CHROMA_DB_DIR)
collection = client.get_collection(COLLECTION_NAME)
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# === FUNCTION: Improved Action Intent Detection ===
def detect_relevant_field(query):
    query = query.lower()
    if any(word in query for word in ["apply", "application", "register", "procedure", "how to avail"]):
        return "Application Process"
    elif "benefit" in query or "advantages" in query:
        return "Benefits"
    elif "eligibility" in query or "eligible" in query or "who can apply" in query:
        return "Eligibility Criteria"
    elif "document" in query or "paperwork" in query:
        return "Documents Required"
    else:
        return None  # No forced field if intent isn't clear

# === FUNCTION: Retrieve Answer Context ===
def retrieve_context(user_query):
    relevant_field = detect_relevant_field(user_query)
    query_embedding = embed_model.encode([user_query])

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=TOP_K
    )

    docs = results['documents'][0]
    metas = results['metadatas'][0]

    # Identify Top-2 unique schemes
    scheme_order = []
    for meta in metas:
        pid = meta['parent_doc_id']
        if pid not in scheme_order:
            scheme_order.append(pid)
        if len(scheme_order) == 2:
            break

    final_context = []

    for idx, pid in enumerate(scheme_order):
        scheme_data = collection.get(where={"parent_doc_id": pid})
        scheme_name = ""
        section = []

        for doc, meta in zip(scheme_data['documents'], scheme_data['metadatas']):
            field = meta['field']
            if field == "Scheme Name" and not scheme_name:
                scheme_name = doc.split(":",1)[1].strip()
                section.append(f"Scheme Name: {scheme_name}")
            if field in CORE_FIELDS:
                section.append(f"{field}: {doc.split(':',1)[1].strip()}")

        # === Action Intent Handling for Top-1 Scheme ===
        if idx == 0 and relevant_field:
            for doc, meta in zip(scheme_data['documents'], scheme_data['metadatas']):
                if meta['field'] == relevant_field:
                    section.append(f"{relevant_field}: {doc.split(':',1)[1].strip()}")

        final_context.append("\n".join(section))

    return "\n\n---\n\n".join(final_context)


In [3]:
retrieve_context("What are the benefits of startup schemes in India?")


'Scheme Name: Bihar Startup Policy\nDescription: This is a policy for entrepreneurs to boost entrepreneurship in the state of Bihar and it was launched in July 2022 .\nBenefits: 1. Bihar’s startup policy was launched in Bihar in the month August 2022 to boost entrepreneurship in the state.\n1. Young Bihar entrepreneurs would get a loan of 10 lacks without interest for ten years. This sum will be allocated as seed money under the Bihar Startup Policy.\n1. Enhanced incentives of 5 per cent for female entrepreneurs, in addition to a grant of 3 lakh rupees for businesses that participate in “rigorous training for product improvement and finance.”\n1. In addition to shared office space, there are financial incentives and other amenities available.\n1. Created a startup platform for young business people in the state of Bihar who is interested in taking advantage of the policies and programs available to them.\n\n---\n\nScheme Name: Startup India Seed Fund Scheme- For Startups\nDescription: 

In [4]:
retrieve_context("How can I apply for the Tripura Industrial Investment Promotion Incentive scheme")

'Scheme Name: Tripura Industrial Investment Promotion Incentive\nDescription: The TIPII scheme aims to attract and promote industrial investments in Tripura by providing various incentives and support to eligible businesses, fostering economic growth and job creation in the state.\nApplication Process: Application ProcessOfflineStep 1: Obtain the application form from the Department of Industries and Commerce or download it from the official website.Step 2: Fill in the form with accurate details regarding the proposed project, investment plans, employment generation, and other required information.Step 3: Attach all necessary documents, including project reports, financial statements, and supporting certificates.Step 4: Submit the completed application form along with the supporting documents to the designated authority within the specified timeframe.Step 5: The application will undergo scrutiny and evaluation by the concerned department, and applicants may be called for a discussion o

In [5]:
retrieve_context("Who is eligible under the Fee Waiver Scheme for SC/ST Students in Gujarat?")

'Scheme Name: Fee Waiver Scheme For SC/ST Students Pursuing Higher Education\nDescription: "Fee Waiver Scheme for SC/ST Students Pursuing Higher Education" provides free access to every SC/ST student, to the institutions of higher learning so that that no eligible SC/ST students are deprived of higher education for want of resources.\nEligibility Criteria: 1. The applicant should have been born in Goa or resident of Goa for at least last fifteen years.\n1. The applicant should have passed his/her qualifying examination from any of the institutions located in Goa.\n1. The applicant should belong to the SC/ST Community of Goa.\n1. The applicant should be enrolled for the First Year of their Degree/PG Courses.\n\n---\n\nScheme Name: Pre-SSC Scholarship for Scheduled Caste Students (Parixitlal Majmudar Scholarship)\nDescription: Under this scheme, scholarships are provided to Scheduled Caste students who are studying in Class 1 to 10 in Government/Grant-in-Aid Schools/Private Schools.'

In [6]:
retrieve_context(" What is the step-by-step process to apply for financial aid under the Widow Pension Scheme of West Bengal?")

'Scheme Name: Widow Pension-West Bengal\nDescription: The West Bengal government\'s Women and Child Development and Social Welfare Department launched the "Widow Pension" scheme in 2010. This program aims to support widows facing economic hardship within the state.\nApplication Process: **Application Form(**[**Form-P)**](https://wbswpension.gov.in/readwrite/miscellaneous/form-P.pdf) **for Pension may be obtained free of cost from following offices namely:**\n1. Office of the Block Development Officer or the Executive Officer of the concerned Panchayat Samity in case of the applicant resides in the rural areas under the said Panchayat Samity.\n1. Office of the Sub-Divisional Officer in case of the applicant resides in Municipal / Notified areas outside the areas of Kolkata Municipal Corporation.\n1. Office of the Controller of Vagrancy, West Bengal situated at Purta Bhawan, Bidhan Nagar, Kolkata-700091, in case of the applicant resides within the areas under Kolkata Municipal Corporatio